In [19]:
import pandas as pd 
import numpy as np

In [ ]:
xx = pd.read_csv(r'C:\Users\tejag\OneDrive\Desktop\A chat bot\Academicpal-ml-chatbot\model\Academicpal chatbot1styear - Sheet1.csv')

In [ ]:
xx.head()

,Semester,Cycle,Subject,Keywords,Notes Link
0,1st,Physics,New Physics,"physics, science, mechanics",https://drive.google.com/drive/folders/1e-LQMg...
1,1st,Physics,Problem Solving,"problem solving, psp, logic",https://drive.google.com/drive/u/5/folders/1yK...
2,1st,Physics,Maths,"math, maths, calculus, algebra",https://drive.google.com/drive/folders/1sYdBua...
3,1st,Physics,Basic Electronics,"electronics, basic electronics, circuits",https://drive.google.com/drive/folders/17iJtHY...
4,1st,Physics,Cyber Security,"cyber security, security, hacking",https://drive.google.com/drive/folders/17kg_R1...


In [ ]:
xx.isnull().sum()

Semester      0
Cycle         0
Subject       0
Keywords      0
Notes Link    0
dtype: int64

In [ ]:
xx["Semester"].value_counts()

Semester
1st    20
Name: count, dtype: int64

In [20]:
pip install nltk fuzzywuzzy pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from fuzzywuzzy import process

# Download NLTK data (only needed once)
nltk.download('punkt')
nltk.download('stopwords')


C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tejag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tejag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import pandas as pd
file_path = "Academicpal chatbot1styear - Sheet1.csv"  # Update with actual path
df = pd.read_csv(file_path)

# Convert to lowercase for better matching
df["Subject"] = df["Subject"].str.lower()
df["Cycle"] = df["Cycle"].str.lower()


In [5]:
def preprocess_input(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)


In [6]:
# Load dataset
file_path = "Academicpal chatbot1styear - Sheet1.csv"  # Update with actual path
df = pd.read_csv(file_path)

# Convert to lowercase for better matching
df["Subject"] = df["Subject"].str.lower()
df["Cycle"] = df["Cycle"].str.lower()


In [7]:
def preprocess_input(text):
    tokens = word_tokenize(text.lower())  # Tokenize & lowercase
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(filtered_tokens)


In [11]:
def get_notes(df):
    user_input = input("What notes do you need? ")  # Example: "I want maths notes"
    processed_input = preprocess_input(user_input)  

    # Use fuzzy matching to find the best subject
    subject_list = df["Subject"].unique()
    best_match, score = process.extractOne(processed_input, subject_list)

    if score < 50:  # If confidence is low
        print("No relevant subject found. Please try again.")
        return

    # Filter subjects that match
    matched_subjects = df[df["Subject"] == best_match]

    # Check if multiple cycles exist
    unique_cycles = matched_subjects["Cycle"].unique()

    if len(unique_cycles) > 1:
        print("Available cycles:", unique_cycles)
        cycle = input("Enter the cycle: ").strip().lower()

        # Filter by cycle
        matched_subjects = matched_subjects[matched_subjects["Cycle"] == cycle]

    # Get the notes link
    if not matched_subjects.empty:
        link = matched_subjects["Notes Link"].values[0]
        print(f"Here is the link to your {best_match} ({cycle}) notes: {link}")
    else:
        print("No notes found for the given subject and cycle.")
get_notes(df)

Available cycles: ['physics' 'chemistry']
Here is the link to your maths (physics) notes: https://drive.google.com/drive/folders/1sYdBua6wr7uhMYw4uIKw5hrPtYd10c_B


In [17]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tejag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\tejag\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tejag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
%pip install textdistance
%pip install pyspellchecker

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from spellchecker import SpellChecker
import pickle
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def correct_spelling(query, keywords):
    spell = SpellChecker()
    # Add domain-specific terms to the spell checker
    spell.word_frequency.load_words(keywords)
    words = query.lower().split()
    corrected_words = []
    for word in words:
        if word in spell:
            corrected_words.append(word)
        else:
            # Find the closest match in keywords
            closest_match = min(keywords, key=lambda k: levenshtein.distance(word, k))
            corrected_words.append(closest_match)
    return " ".join(corrected_words)

def load_data():
    data = pd.read_csv(r'C:\Users\tejag\OneDrive\Desktop\A chat bot\Academicpal-ml-chatbot\model\Academicpal chatbot1styear - Sheet1.csv')
    # Validate data
    if data.isnull().any().any():
        logging.warning("Missing values found in the dataset. Filling with empty strings.")
        data = data.fillna('')
    return data

def train_model(data):
    # Combine relevant columns into a single text column
    data['Text'] = data['Cycle'] + " " + data['Subject'] + " " + data['Keywords']
    X = data['Text']
    y = data['Notes Link']
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Define pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(lowercase=True, stop_words='english')),
        ('clf', RandomForestClassifier(random_state=42))
    ])
    
    # Define hyperparameters for grid search
    parameters = {
        'tfidf__max_df': (0.5, 0.75, 1.0),
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'clf__n_estimators': [10, 50, 100],
        'clf__max_depth': [None, 10, 20],
    }
    
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    
    # Log best parameters and score
    logging.info("Best parameters found: %s", grid_search.best_params_)
    logging.info("Best cross-validation score: %s", grid_search.best_score_)
    
    # Evaluate on the test set
    y_pred = grid_search.predict(X_test)
    logging.info("Classification Report:\n%s", classification_report(y_test, y_pred))
    
    # Save the vectorizer and model separately
    vectorizer = grid_search.best_estimator_.named_steps['tfidf']
    model = grid_search.best_estimator_.named_steps['clf']
    
    with open("vectorizer.pkl", "wb") as vectorizer_file:
        pickle.dump(vectorizer, vectorizer_file)
    
    with open("model.pkl", "wb") as model_file:
        pickle.dump(model, model_file)
    
    # Save keywords
    with open("keywords.pkl", "wb") as keywords_file:
        pickle.dump(data['Keywords'].str.split(', ').explode().unique(), keywords_file)

def find_notes(query):
    try:
        if not query.strip():
            logging.warning("Empty query provided.")
            return None
        
        # Load vectorizer, model, and keywords
        with open("vectorizer.pkl", "rb") as vectorizer_file:
            vectorizer = pickle.load(vectorizer_file)
        
        with open("model.pkl", "rb") as model_file:
            model = pickle.load(model_file)
        
        with open("keywords.pkl", "rb") as keywords_file:
            keywords = pickle.load(keywords_file)
        
        # Correct spelling
        corrected_query = correct_spelling(query, keywords)
        logging.info("Corrected query: %s", corrected_query)
        
        # Transform the query using the vectorizer
        query_vector = vectorizer.transform([corrected_query])
        
        # Predict the notes link
        prediction = model.predict(query_vector)[0]
        return prediction
    except Exception as e:
        logging.error("An error occurred: %s", e)
        return None

# Load dataset
data = load_data()
train_model(data)

# Example query
query = "I want mathz notes from phyzics cycle"
notes_link = find_notes(query)
if notes_link:
    print("Best matching notes link:", notes_link)
else:
    print("Failed to find notes link.")



Fitting 2 folds for each of 54 candidates, totalling 108 fits


C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
2025-02-27 09:42:59,424 - INFO - Best parameters found: {'clf__max_depth': None, 'clf__n_estimators': 10, 'tfidf__max_df': 0.5, 'tfidf__ngram_range': (1, 1)}
2025-02-27 09:42:59,425 - INFO - Best cross-validation score: 0.125
C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Pytho

Best matching notes link: https://drive.google.com/drive/folders/1sYdBua6wr7uhMYw4uIKw5hrPtYd10c_B


In [9]:
query = "I want mathz notes from phyzics cycle"
notes_link = find_notes(query)
if notes_link:
    print("Best matching notes link:", notes_link)
else:
    print("Failed to find notes link.")

2025-02-27 09:44:46,049 - INFO - Corrected query: i want math notes from physics cycle


Best matching notes link: https://drive.google.com/drive/folders/1sYdBua6wr7uhMYw4uIKw5hrPtYd10c_B


In [2]:
# Modified model training code
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from spellchecker import SpellChecker
import textdistance
import pickle
import logging
import numpy as np

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def correct_spelling(query, keywords):
    spell = SpellChecker()
    # Add domain-specific terms to the spell checker
    spell.word_frequency.load_words(keywords)
    words = query.lower().split()
    corrected_words = []
    for word in words:
        if word in spell:
            corrected_words.append(word)
        else:
            # Use a safer approach to find closest match
            min_distance = float('inf')
            closest_word = word  # Default to original word
            
            for keyword in keywords:
                try:
                    distance = textdistance.levenshtein.distance(word, keyword)
                    # Handle case where distance might be an array
                    if isinstance(distance, (list, np.ndarray)):
                        distance = float(distance[0]) if len(distance) > 0 else float('inf')
                    
                    if distance < min_distance:
                        min_distance = distance
                        closest_word = keyword
                except Exception as e:
                    logging.error(f"Error calculating distance: {e}")
                    continue
            
            corrected_words.append(closest_word)
            
    return " ".join(corrected_words)

def load_data():
    data = pd.read_csv(r'C:\Users\tejag\OneDrive\Desktop\A chat bot\Academicpal-ml-chatbot\Academicpal chatbot1styear - Sheet1.csv')
    # Validate data
    if data.isnull().any().any():
        logging.warning("Missing values found in the dataset. Filling with empty strings.")
        data = data.fillna('')
    return data

def train_model(data):
    # Combine relevant columns into a single text column
    data['Text'] = data['Cycle'] + " " + data['Subject'] + " " + data['Keywords']
    X = data['Text']
    y = data['Notes Link']
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Define pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(lowercase=True, stop_words='english')),
        ('clf', RandomForestClassifier(random_state=42))
    ])
    
    # Define hyperparameters for grid search
    parameters = {
        'tfidf__max_df': (0.5, 0.75, 1.0),
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'clf__n_estimators': [10, 50, 100],
        'clf__max_depth': [None, 10, 20],
    }
    
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    
    # Log best parameters and score
    logging.info("Best parameters found: %s", grid_search.best_params_)
    logging.info("Best cross-validation score: %s", grid_search.best_score_)
    
    # Evaluate on the test set
    y_pred = grid_search.predict(X_test)
    logging.info("Classification Report:\n%s", classification_report(y_test, y_pred))
    
    # Save the vectorizer and model separately
    vectorizer = grid_search.best_estimator_.named_steps['tfidf']
    model = grid_search.best_estimator_.named_steps['clf']
    
    with open("vectorizer.pkl", "wb") as vectorizer_file:
        pickle.dump(vectorizer, vectorizer_file)
    
    with open("model.pkl", "wb") as model_file:
        pickle.dump(model, model_file)
    
    # Save keywords - make sure they're plain strings, not numpy arrays or other complex types
    keywords_list = data['Keywords'].str.split(', ').explode().unique().tolist()
    with open("keywords.pkl", "wb") as keywords_file:
        pickle.dump(keywords_list, keywords_file)

def find_notes(query):
    try:
        if not query.strip():
            logging.warning("Empty query provided.")
            return None
        
        # Load vectorizer, model, and keywords
        with open("vectorizer.pkl", "rb") as vectorizer_file:
            vectorizer = pickle.load(vectorizer_file)
        
        with open("model.pkl", "rb") as model_file:
            model = pickle.load(model_file)
        
        with open("keywords.pkl", "rb") as keywords_file:
            keywords = pickle.load(keywords_file)
        
        # Correct spelling
        corrected_query = correct_spelling(query, keywords)
        logging.info("Corrected query: %s", corrected_query)
        
        # Transform the query using the vectorizer
        query_vector = vectorizer.transform([corrected_query])
        
        # Predict the notes link
        prediction = model.predict(query_vector)[0]
        return prediction
    except Exception as e:
        logging.error("An error occurred: %s", e)
        return None

# Main execution
if __name__ == "__main__":
    # Load dataset
    data = load_data()
    train_model(data)

    # Example query
    query = "I want mathz notes from phyzics cycle"
    notes_link = find_notes(query)
    if notes_link:
        print("Best matching notes link:", notes_link)
    else:
        print("Failed to find notes link.")

Fitting 2 folds for each of 54 candidates, totalling 108 fits


C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
2025-02-27 10:22:50,848 - INFO - Best parameters found: {'clf__max_depth': None, 'clf__n_estimators': 10, 'tfidf__max_df': 0.5, 'tfidf__ngram_range': (1, 1)}
2025-02-27 10:22:50,849 - INFO - Best cross-validation score: 0.125
C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Pytho

Best matching notes link: https://drive.google.com/drive/folders/1sYdBua6wr7uhMYw4uIKw5hrPtYd10c_B
